In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
with open('quote.txt', 'r', encoding='utf-8') as f:
    txt_content = f.read()

In [ ]:
! pip install PyPDF2
! pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.8 MB/s eta 0:00:00


In [ ]:
# import PyPDF2
# from math import ceil

# pdf_content = ""
# with open('Pride_and_Prejudice.pdf', 'rb') as f:
#     reader = PyPDF2.PdfReader(f)
#     total_pages = len(reader.pages)
#     partial_pages = ceil(total_pages // 64)  # 1/16th of the total pages

#     for i in range(partial_pages):
#         text = reader.pages[i].extract_text()
#         if text:
#             pdf_content += text


In [ ]:
# import docx

# def read_docx(filename):
#     doc = docx.Document(filename)
#     return "\n".join([para.text for para in doc.paragraphs])

# docx_content = read_docx('Sasuke.docx')

In [ ]:
# text = pdf_content
text = txt_content

In [ ]:
chars = sorted(list(set(text)))  # Extract unique characters from the text
char_to_index = {char: i for i, char in enumerate(chars)}  # Map characters to indices
index_to_char = {i: char for i, char in enumerate(chars)}

In [ ]:
seq_length = 3
sequences = []
labels = []

In [ ]:
for i in range(len(text) - seq_length):
    seq = text[i:i + seq_length]  # Sequence of characters
    label = text[i + seq_length]  # The next character after the sequence
    sequences.append([char_to_index[char] for char in seq])  # Convert each char to its index
    labels.append(char_to_index[label])  # Convert the label char to its index

In [ ]:
x = np.array(sequences)
y = np.array(labels)

In [ ]:
x_tensor = torch.from_numpy(x)
y_tensor = torch.from_numpy(y)

In [ ]:
x_one_hot = torch.nn.functional.one_hot(x_tensor, num_classes = len(chars)).float()

In [ ]:
dataset = TensorDataset(x_one_hot, y_tensor)
batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class CharLSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
     super(CharLSTM, self).__init__()
     self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
     self.fc = nn.Linear(hidden_size, num_classes)
  def forward(self, x):
     out, _ = self.lstm(x)
     out = out[:, -1, :]
     out = self.fc(out)
     return out


In [ ]:
input_size = len(chars)
hidden_size = 160
num_layers = 6
num_classes = len(chars)
num_epochs = 200
learning_rate = 0.001

In [ ]:
model = CharLSTM(input_size, hidden_size, num_layers, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(num_epochs):
  total_loss = 0
  for batch_x, batch_y in dataloader:
    optimizer.zero_grad()
    outputs = model(batch_x)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  avg_loss = total_loss/len(dataloader)
  print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss: .4f}")

Epoch 1/200, Loss:  3.3298
Epoch 2/200, Loss:  3.2619
Epoch 3/200, Loss:  3.2003
Epoch 4/200, Loss:  3.1081
Epoch 5/200, Loss:  2.9668
Epoch 6/200, Loss:  2.8504
Epoch 7/200, Loss:  2.7658
Epoch 8/200, Loss:  2.6710
Epoch 9/200, Loss:  2.5901
Epoch 10/200, Loss:  2.5412
Epoch 11/200, Loss:  2.4706
Epoch 12/200, Loss:  2.4258
Epoch 13/200, Loss:  2.3728
Epoch 14/200, Loss:  2.3351
Epoch 15/200, Loss:  2.2730
Epoch 16/200, Loss:  2.2231
Epoch 17/200, Loss:  2.1763
Epoch 18/200, Loss:  2.1445
Epoch 19/200, Loss:  2.0883
Epoch 20/200, Loss:  2.0701
Epoch 21/200, Loss:  2.0145
Epoch 22/200, Loss:  1.9728
Epoch 23/200, Loss:  1.9291
Epoch 24/200, Loss:  1.9065
Epoch 25/200, Loss:  1.8525
Epoch 26/200, Loss:  1.8255
Epoch 27/200, Loss:  1.8027
Epoch 28/200, Loss:  1.7505
Epoch 29/200, Loss:  1.7274
Epoch 30/200, Loss:  1.7098
Epoch 31/200, Loss:  1.6732
Epoch 32/200, Loss:  1.6313
Epoch 33/200, Loss:  1.6120
Epoch 34/200, Loss:  1.5998
Epoch 35/200, Loss:  1.5676
Epoch 36/200, Loss:  1.5429
E

In [ ]:
def generate_text(model, start_seq, length, char_to_index, index_to_char):
  model.eval()
  seq = [char_to_index[c] for c in start_seq]
  generated = start_seq
  for _ in range(length):
    x = torch.tensor([seq[-seq_length:]])
    x_onehot = torch.nn.functional.one_hot(x, num_classes=len(chars)).float()
    with torch.no_grad():
      out = model(x_onehot)
      pred = out.argmax(dim=1).item()
    generated += index_to_char[pred]
    seq.append(pred)
  return generated

In [ ]:
print("Generated Text: ")
print(generate_text(model, "Once", 100, char_to_index, index_to_char))

Generated Text: 
Once and still as if you are to be may not live world be may not live world be may not live world be may


# Exporting Model and Vocab

In [ ]:
import torch
import pickle

# Save model
torch.save(model.state_dict(), "text_generator_model.pth")

# Save vocab and mappings
with open("vocab.pkl", "wb") as f:
    # Save the list of unique characters, which represents the vocabulary
    pickle.dump(chars, f)

with open("char_to_idx.pkl", "wb") as f:
    pickle.dump(char_to_index, f) # Changed char_to_idx to char_to_index

with open("idx_to_char.pkl", "wb") as f:
    pickle.dump(index_to_char, f) # Changed idx_to_char to index_to_char

print("✅ Model and mappings exported successfully.")

✅ Model and mappings exported successfully.


# Text Generation Function

In [ ]:
torch.save(model.state_dict(), "text_generator_model.pth")
with open("vocab.pkl", "wb") as f:
    pickle.dump(chars, f)
with open("char_to_idx.pkl", "wb") as f:
    pickle.dump(char_to_index, f)
with open("idx_to_char.pkl", "wb") as f:
    pickle.dump(index_to_char, f)

# CLI Support

In [ ]:
if __name__ == "__main__":
    prompt = input("Enter a blog title or starting text: ")
    print("\nGenerated Blog:\n")
    # Call generate_text with all required arguments
    print(generate_text(model, prompt, 100, char_to_index, index_to_char))

Enter a blog title or starting text: love

Generated Blog:

love world be may not live world be may not live world be may not live world be may not live world be ma
